In [ ]:
!pip install --quiet --upgrade wrds pandas psycopg2-binary

!pip install --quiet arch statsmodels openpyxl wrds


In [ ]:
import wrds, pandas as pd, numpy as np
from google.colab import files
import pandas as pd, numpy as np, os, datetime as dt
from pathlib import Path

In [ ]:

db = wrds.Connection()



In [ ]:
files.upload()

files.upload()

In [ ]:

# Levels → Returns panel,

# ---------------------------------------------------------------------------
# 0 ·  Paths
# ---------------------------------------------------------------------------
SECTOR_FILE = '/content/sp500_sector_indices.csv'
CLOSE_FILE  = '/content/sp500_close_daily.csv'

OUT_DIR = os.path.dirname(SECTOR_FILE) or '/content'
os.makedirs(OUT_DIR, exist_ok=True)

# ---------------------------------------------------------------------------
# 1 ·  Read CSVs
# ---------------------------------------------------------------------------
sector_lvl = (pd.read_csv(SECTOR_FILE, parse_dates=['date'])
                .set_index('date')
                .sort_index())

spx_close  = (pd.read_csv(CLOSE_FILE, parse_dates=['date'])
                .set_index('date')
                .sort_index())

# ---------------------------------------------------------------------------
# 2 ·  Restrict to 1991-01-01 … 2024-11-01 (inclusive)
# ---------------------------------------------------------------------------
START = pd.Timestamp('1991-01-01')
END   = pd.Timestamp('2024-11-01')

sector_lvl = sector_lvl.loc[START:END]
spx_close  = spx_close.loc[START:END]

# align on union of trading days
lvl_panel = sector_lvl.join(spx_close, how='outer')

# ---------------------------------------------------------------------------
# 3 ·  Log-returns in percent
# ---------------------------------------------------------------------------
ret_panel = (np.log(lvl_panel).diff()
             .mul(100)
             .rename(columns={'sp500_close': 'SPX_logret'}))

# ---------------------------------------------------------------------------
# 4 ·  Save
# ---------------------------------------------------------------------------
#jeg lod oprindeligt min data løbe ind i 2025 hvilket er for langt det rettede jeg men fik ikke ændret i navnene
lvl_out = os.path.join(OUT_DIR, 'sp500_sector_indices_merged_1991_2025.csv')
ret_out = os.path.join(OUT_DIR, 'sp500_sector_returns_panel_1991_2025.csv')

lvl_panel.to_csv(lvl_out)
ret_panel.to_csv(ret_out)



In [ ]:

# Build ONE master CSV (levels + returns + RF)



# ── paths (edit if needed) ──────────────────────────────────────────────────
lvl_path    = '/content/sp500_sector_indices_merged_1991_2025.csv'
ret_path    = '/content/sp500_sector_returns_panel_1991_2025.csv'
master_path = '/content/sp500_sector_master_1991_2025.csv'

# ── read levels & returns ───────────────────────────────────────────────────
lvl_df = pd.read_csv(lvl_path, index_col=0, parse_dates=True)
ret_df = pd.read_csv(ret_path, index_col=0, parse_dates=True)

# ── add suffixes so names are unique ────────────────────────────────────────
lvl_df = lvl_df.add_suffix('_lvl')                     # Energy_lvl, …

ret_df = ret_df.rename(                                # Energy_ret, …
    columns=lambda c: c if c == 'RF_log' else f'{c}_ret')

# ── join on the date index ──────────────────────────────────────────────────
master = lvl_df.join(ret_df, how='outer')              # no more collision

# ── save & trigger download ────────────────────────────────────────────────
master.to_csv(master_path)
print(f"Master CSV saved ➜ {master_path}")
print("Shape:", master.shape)

files.download(master_path)



In [ ]:

# ------------------------------------------------------------
# 1) download the two columns
# ------------------------------------------------------------
ff = db.get_table('ff', 'factors_daily', columns=['date', 'rf'])

# ------------------------------------------------------------
# 2) robust date parsing
#    - handles integers (19260701) AND strings ('1926-07-01')
# ------------------------------------------------------------
ff['date'] = pd.to_datetime(ff['date'], errors='coerce')

# drop rows that failed to parse, sort, and keep one per day
ff = (ff.dropna(subset=['date'])
        .sort_values('date')
        .drop_duplicates('date', keep='last')
        .set_index('date'))

# ------------------------------------------------------------
# 3) convert to continuously-compounded daily rate
#    rf is in *percent* daily; divide by 100, then log(1+rf)
# ------------------------------------------------------------
ff['RF_log'] = np.log1p(ff['rf'] / 100)

# ------------------------------------------------------------
# 4) align exactly to the trading-day index in your master panel
#    (missing dates – normally the latest 4-5 weeks – stay NaN)
# ------------------------------------------------------------
df = df.copy()
df['RF_log'] = ff['RF_log'].reindex(df.index)

# ------------------------------------------------------------
# 5) save back to the same CSV (overwrite) and show a quick check
# ------------------------------------------------------------
df.to_csv(MASTER)
non_na = df['RF_log'].notna().sum()



In [ ]:

files.download(MASTER)


In [ ]:
import pandas as pd
from pathlib import Path

# ----- 1.  Define the calendar data ---------------------------------
calendar = {
    "election":   [1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020],
    "call_date":  [
        "1992-11-04", "1996-11-06", "2000-11-08", "2004-11-03",
        "2008-11-05", "2012-11-07", "2016-11-09", "2020-11-05"
    ],
    "inaug_date": [
        "1993-01-20", "1997-01-20", "2001-01-22", "2005-01-20",
        "2009-01-20", "2013-01-22", "2017-01-20", "2021-01-20"
    ],
    "sectors":    [
        "XLI|XLF|XLV",  # 1992 Clinton
        "XLK|XLP|XLC",  # 1996 Clinton
        "XLF|XLK|XLE",  # 2000 Bush
        "XLE|XLI|XLF",  # 2004 Bush
        "XLU|XLV|XLF",  # 2008 Obama
        "XLE|XLU|XLV",  # 2012 Obama
        "XLE|XLF|XLI",  # 2016 Trump
        "XLU|XLI|XLV"   # 2020 Biden
    ]
}

# ----- 2.  Build a DataFrame ----------------------------------------
cal_df = pd.DataFrame(calendar)

# ----- 3.  Save to CSV -------------
output_path = Path("election_calendar.csv")
cal_df.to_csv(output_path, index=False, encoding="utf-8")


